<a href="https://colab.research.google.com/github/pujangrg1/pujan_INFO5731_Spring2020/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

### **Install Packages**

In [2]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor

     |████████████████████████████████| 133kB 8.6MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52557 sha256=62950b7f30ea6ae3d81d598682ba7bdabbe55746609b8477bf5aed7ab302a41c
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 235kB 8.2MB/s 
  Created wheel for pyicu: filename=PyICU-2.6-cp37-cp37m-linux_x86_64.whl size=1306379 sha256=affe2664d9ea2c7681531fc7c4f2a72186bc7af7b09d0e782060670d6714d524
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
Successfully built pyicu
     |████████████████████████████████| 41.4MB 100kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834244 sha256=dc6f114ff55b411273d68591c93baa3bcca8e8d13e8807752d4a5379c7bfe21d
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3d

In [3]:
%%bash
polyglot download sentiment2.en

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...


## **(1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.**

### **Load and prepare the dataset**

In [4]:
import nltk
nltk.download('stopwords')
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/pujangrg1/pujan_INFO5731_Spring2020/main/MovieReviews.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### **Preprocess Dataset**

In [5]:
from nltk.corpus import stopwords
from textblob import TextBlob
df['Review'] = df['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Review'] = df['Review'].str.replace('[^\w\s]','')
text = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in text))
df['Review'] = df['Review'].apply(lambda x: str(TextBlob(x).correct()))

### **Extract Words related to Sentiment using Polyglot**

In [7]:
from polyglot.text import Text
sentiment_words = []
for sentence in df['Review']:
  word = Text(sentence)
  for w in word.words:
    if w.polarity != 0:
      sentiment_words.append(w)
sentiment_words

['brilliance',
 'grotesque',
 'hard',
 'mesmerizing',
 'tragic',
 'funny',
 'rollercoaster',
 'proper',
 'best',
 'isolated',
 'sorry',
 'encourage',
 'better',
 'respect',
 'like',
 'isolated',
 'masterpiece',
 'best',
 'best',
 'best',
 'willingly',
 'remarkable',
 'disturbing',
 'violent',
 'proving',
 'corruption',
 'rich',
 'talented',
 'believable',
 'shake',
 'bitter',
 'comforting',
 'perfect',
 'fearless',
 'stunning',
 'impossible',
 'dark',
 'joke',
 'stunned',
 'accomplished',
 'impossible',
 'exceeds',
 'dark',
 'crime',
 'like',
 'falling',
 'joke',
 'happy',
 'ignore',
 'complaints',
 'pernicious',
 'embarrassing',
 'better',
 'criminal',
 'dark',
 'joke',
 'dark',
 'fall',
 'love',
 'bad',
 'romantic',
 'scars',
 'amazing',
 'best',
 'breathtaking',
 'uncomfortable',
 'like',
 'horrible',
 'great',
 'inspiration',
 'honest',
 'violent',
 'disturbing',
 'violent',
 'controversy',
 'great',
 'best',
 'hate',
 'madness',
 'enjoy',
 'masterpiece',
 'amazing',
 'enjoy',
 'jo

### **Rank Sentiment related words by frequency**

In [8]:
from collections import Counter
freq = Counter(sentiment_words)
df_rank = pd.DataFrame(list(freq.most_common()), columns=['Words', 'Frequency'])
df_rank.index = list(range(1, len(freq.most_common())+1))
df_rank

,Words,Frequency
1,joke,135
2,like,61
3,good,53
4,best,40
5,dark,33
...,...,...
467,cry,1
468,marvelous,1
469,tolerable,1
470,pleasantly,1


## **(2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers.**

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [12]:
# Write your code here
def Encoder(i):
  sentiment_encoded = 0
  if i == 'Positive':
    sentiment_encoded = 1
  elif i == 'Negative':
    sentiment_encoded = 2
  return sentiment_encoded


In [13]:
actual_sentiment = []
for i in df['Sentimental Analysis']:
  actual_sentiment.append(Encoder(i))

### **Sentimental Aalysis with TextBlob**

In [14]:
from textblob import TextBlob
tb_sentiments = []
for sen in df['Review']:
  p = TextBlob(sen).sentiment.polarity
  if p > 0:
    tb_sentiments.append(1)
  elif p < 0:
    tb_sentiments.append(2)
  elif p == 0.0:
    tb_sentiments.append(0)
tb_df = pd.DataFrame(list(zip(df['Review'], actual_sentiment, tb_sentiments)), columns = ['Reviews', 'Actual', 'Predicted'])
tb_df

,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,2
1,movie felt alone isolated truly relate underst...,1,0
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,2
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,2
105,plan watching would suggest shipping last 20 3...,2,2
106,want title make wouldn know based comicbook ch...,1,1


In [17]:
from sklearn.metrics import f1_score, accuracy_score
accuracy_of_textblob = accuracy_score(tb_df['Actual'], tb_df['Predicted'])*100
f1Score_of_textblob = f1_score(tb_df['Actual'], tb_df['Predicted'], average = 'macro')
print("Text Blob Accuracy is {0}\nF1 Score is {1}".format(accuracy_of_textblob, f1Score_of_textblob))

Text Blob Accuracy is 56.481481481481474
F1 Score is 0.399400871459695


### **Sentimental Aalysis with SVM**

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [20]:
train_x, test_x, train_y, test_y = train_test_split(df['Review'],df['Sentimental Analysis'],test_size=0.33)
Encoder = LabelEncoder()
train_y = Encoder.fit_transform(train_y)
test_y = Encoder.fit_transform(test_y)
train_x_vectors = tfid_vectorizer.fit_transform(train_x)
test_x_vectors = tfid_vectorizer.transform(test_x)

In [21]:
from sklearn import svm
from sklearn.metrics import classification_report
svm_model = svm.SVC(kernel='linear')
svm_model.fit(train_x_vectors, train_y)
predicted = svm_model.predict(test_x_vectors)
summary = classification_report(test_y, predicted, output_dict=True)
summary

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'f1-score': 0.39999999999999997,
  'precision': 0.75,
  'recall': 0.2727272727272727,
  'support': 11},
 '1': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 5},
 '2': {'f1-score': 0.7307692307692308,
  'precision': 0.59375,
  'recall': 0.95,
  'support': 20},
 'accuracy': 0.6111111111111112,
 'macro avg': {'f1-score': 0.3769230769230769,
  'precision': 0.4479166666666667,
  'recall': 0.40757575757575754,
  'support': 36},
 'weighted avg': {'f1-score': 0.5282051282051282,
  'precision': 0.5590277777777778,
  'recall': 0.6111111111111112,
  'support': 36}}

### **Sentimental Aalysis with Vader**

In [22]:
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()
vader_senti = []
for line in df['Review']:
  polarity = vader.polarity_scores(line)['compound']
  if polarity > 0:
    vader_senti.append(1)
  elif polarity < 0:
    vader_senti.append(2)
  elif polarity == 0.0:
    vader_senti.append(0)
df_vader = pd.DataFrame(list(zip(df['Review'], actual_sentiment, vader_senti)), columns = ['Reviews', 'Actual', 'Predicted'])
df_vader

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


,Reviews,Actual,Predicted
0,every movie comes truly makes impact joaquins ...,1,1
1,movie felt alone isolated truly relate underst...,1,1
2,truly masterpiece best hollywood film 2019 one...,1,1
3,joaquin phoenix gives tour de force performanc...,0,1
4,time moves anticipated like end falling short ...,2,1
...,...,...,...
103,truly masterpiece best hollywood film 2019 one...,1,1
104,acting cinematography sound design script phen...,1,1
105,plan watching would suggest shipping last 20 3...,2,1
106,want title make wouldn know based comicbook ch...,1,2


In [24]:
from sklearn.metrics import f1_score, accuracy_score
accuracy_of_vader = accuracy_score(df_vader['Actual'], df_vader['Predicted'])*100
f1Score_of_vader = f1_score(df_vader['Actual'], df_vader['Predicted'], average = 'macro')
print("Vader Accuracy is {0}\nF1 Score is {1}".format(accuracy_of_vader, f1Score_of_vader))

Vader Accuracy is 58.333333333333336
F1 Score is 0.3673289183222958


### **Analysis**

**Accuracy F1-Score**
**TextBlob**      56.48     39.94
**SVM**           61.12     40.00
**Vader**         58.33     36.73


In [29]:
print('           Accuracy  F1-Score')
print('TextBlob     56.48     39.94')
print('SVM          61.12     40.00')
print('Vader        58.33     36.73')
print("\nWe can see from the result above that SVM has the highest F1-Score amongst all the rest.\nThis proves that SVM is better compared to TextBlob and Vader.")

           Accuracy  F1-Score
TextBlob     56.48     39.94
SVM          61.12     40.00
Vader        58.33     36.73

We can see from the result above that SVM has the highest F1-Score amongst all the rest.
This proves that SVM is better compared to TextBlob and Vader.
